Once you are ready to divide the workbooks into individual tables, run the following script to divide your updated workbooks into Excel files which contain a single table of data. This will mean that another script ("HIDataAutoEmail.ipynb") will be able to accurately email only relevant tables to specific contacts.

Remember to adjust the filepaths first.

In [ ]:
import os
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from copy import copy

def copy_cell_attributes(source_cell, target_cell):
    """
    Copy the attributes from the source cell to the target cell, including value, style, and more.
    """
    target_cell.value = source_cell.value
    
    # Copying style attributes, avoiding unhashable types
    if source_cell.has_style:
        target_cell.font = copy(source_cell.font)
        target_cell.border = copy(source_cell.border)
        target_cell.fill = copy(source_cell.fill)
        target_cell.number_format = copy(source_cell.number_format)
        target_cell.protection = copy(source_cell.protection)
        target_cell.alignment = copy(source_cell.alignment)

def extract_sheets(input_file, output_directory):
    # Load the input workbook
    wb = openpyxl.load_workbook(input_file)

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Iterate over sheets in the input workbook
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Check if the sheet is visible
        if sheet.sheet_state == "visible":
            # Create a new workbook for each sheet
            new_wb = Workbook()
            new_sheet = new_wb.active
            new_sheet.title = sheet_name

            # Copy the entire content of the sheet
            for row_idx, row in enumerate(sheet.iter_rows(), start=1):
                for col_idx, cell in enumerate(row, start=1):
                    new_cell = new_sheet.cell(row=row_idx, column=col_idx)
                    copy_cell_attributes(cell, new_cell)

            # Copy tables
            for table in sheet.tables.values():
                # Define the range for the table
                table_range = table.ref

                # Create a new table in the new sheet
                new_table = Table(displayName=table.displayName, ref=table_range)

                # Copy table style
                if table.tableStyleInfo:
                    new_style = TableStyleInfo(
                        name=table.tableStyleInfo.name,
                        showFirstColumn=table.tableStyleInfo.showFirstColumn,
                        showLastColumn=table.tableStyleInfo.showLastColumn,
                        showRowStripes=table.tableStyleInfo.showRowStripes,
                        showColumnStripes=table.tableStyleInfo.showColumnStripes
                    )
                    new_table.tableStyleInfo = new_style

                new_sheet.add_table(new_table)

            # Define the output file path
            output_file = os.path.join(output_directory, f"{sheet_name}.xlsx")
            
            # Save the new workbook with the sheet name as the file name
            new_wb.save(output_file)

def process_folder(input_folder, output_directory):
    """
    Process all Excel files in the input folder and extract sheets from each file.
    """
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".xlsx"):  # Only process .xlsx files
            input_file = os.path.join(input_folder, filename)
            # Create a subfolder for each input file based on the file name (without extension)
            file_output_directory = os.path.join(output_directory, os.path.splitext(filename)[0])
            os.makedirs(file_output_directory, exist_ok=True)

            # Extract sheets from the current file
            extract_sheets(input_file, file_output_directory)

if __name__ == "__main__":
    # Replace 'input_folder' with the path to the folder containing your Excel files
    input_folder = r'C:\filepath\Updated HI Files'

    # Replace 'output_directory' with the desired output directory for extracted sheets
    output_directory = r'C:\filepath\Updated HI Files Split Tables'
    
    # Process all Excel files in the input folder
    process_folder(input_folder, output_directory)
    print("Process complete, new folders containing individual excel sheets for all tables have been created in the 'output_directory'.")
